# Question 1

Given some sample data, write a program to answer the following: click here to access the required data set (https://docs.google.com/spreadsheets/d/16i38oonuX1y1g7C_UAmiK9GkY7cS-64DfiDMNiR41LM/edit#gid=0)

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis.

Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
What metric would you report for this dataset?
What is its value?


In [119]:
import pandas as pd
import numpy as np

In [120]:
sales_df = pd.read_csv("sales_data.csv")

In [121]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        5000 non-null   int64 
 1   shop_id         5000 non-null   int64 
 2   user_id         5000 non-null   int64 
 3   order_amount    5000 non-null   int64 
 4   total_items     5000 non-null   int64 
 5   payment_method  5000 non-null   object
 6   created_at      5000 non-null   object
dtypes: int64(5), object(2)
memory usage: 273.6+ KB


First, I will take a look at the dataset with some samples to get a rough idea of the structure of the dataset. 

In [122]:
#visualize the data
sales_df.sample(20)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
1940,1941,46,916,166,1,debit,2017-03-26 19:16:48
2379,2380,24,980,280,2,debit,2017-03-08 18:46:33
2584,2585,89,700,392,2,cash,2017-03-21 12:18:52
2300,2301,82,764,531,3,credit_card,2017-03-27 23:29:38
3176,3177,96,856,459,3,debit,2017-03-01 7:47:32
1544,1545,34,794,122,1,credit_card,2017-03-24 11:41:02
3249,3250,81,941,354,2,debit,2017-03-11 14:09:22
1315,1316,86,763,260,2,cash,2017-03-06 1:32:05
1312,1313,82,927,531,3,credit_card,2017-03-20 8:32:00
2484,2485,8,996,528,4,debit,2017-03-09 17:26:20


## Naive AOV

Given the data are already in a 30 day window (March), the naive way to calculate AOV is to divide the revenue by the number of orders.

In [123]:
Naive_AOV_30 = sales_df.order_amount.sum() / sales_df.order_id.count()

In [124]:
Naive_AOV_30

3145.128

However, if we look at the 20 samples from above, none of the order_amount value are close to \\$3145.13. To further investigate, we shall look at the highest order_amount in the data, and see if there is skewness.

In [75]:
sales_df.skew(axis=0)

order_id         0.000000
shop_id          0.013830
user_id         -0.034052
order_amount    16.675033
total_items     17.065556
dtype: float64

Both the order_amount and total_items are pretty positively skewed, we shall take a look the top order_amount and total_items.

In [125]:
sales_df.sort_values(['order_amount','total_items'], ascending=False).head(50)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00:00
1362,1363,42,607,704000,2000,credit_card,2017-03-15 4:00:00
1436,1437,42,607,704000,2000,credit_card,2017-03-11 4:00:00
1562,1563,42,607,704000,2000,credit_card,2017-03-19 4:00:00
1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00
2153,2154,42,607,704000,2000,credit_card,2017-03-12 4:00:00
2297,2298,42,607,704000,2000,credit_card,2017-03-07 4:00:00


Next, just by looking at the data returned, there is one customer user_id 607 which bought 2000 items from the same shop in mutiple orders, while most orders are under 6 items. Also, the price for one sneaker in shop_id 78 is \\$25725, which does not make any sense given that a sneaker is a relatively afforable item. This is the reason  why our AOV is so high. To obtain a more reasonable AOV, we shall drop the all the outlier, tuples with total_items = 2000, and the tuples where 1 sneaker is not priced reasonable, ie. shop_id 78 with the sneaker priced at \\$25725.

In [97]:
index = sales_df[(sales_df['order_amount'] >= 25725)].index
sales_df.drop(index, inplace = True)

In [100]:
AOV_30 = sales_df.order_amount.sum() / sales_df.order_id.count()
print(AOV_30)

302.58051448247926


After some data cleaning, the new AOV is \\$302.58.

## Using median aggregator instead of average

Apart from cleaning the data by dropping some outliers, we can also apply median which return the central stendency for skewed distributions.

In [108]:
sales_df = pd.read_csv("sales_data.csv")

In [113]:
sales_df.order_amount.median()

284.0

With median, we get \\$284 even with the skewed data remain in the dataset. 

# Question 2

For this question you’ll need to use SQL. Follow this link (https://www.w3schools.com/SQL/TRYSQL.ASP?FILENAME=TRYSQL_SELECT_ALL) to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.

**a. How many orders were shipped by Speedy Express in total?**

SQL Query:  SELECT count(*) FROM Orders LEFT JOIN Shippers ON Orders.ShipperID = Shippers.ShipperID WHERE ShipperName='Speedy Express';

<img src="files/a.png">

54 orders were shipped by Speedy Express.

**b. What is the last name of the employee with the most orders?**

SQL Query:  SELECT TOP 1 count(Employees.EmployeeID) as number_of_orders, LastName FROM Orders LEFT JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID GROUP BY Employees.EmployeeID, LastName ORDER BY count(Employees.EmployeeID) DESC;

<img src="files/b.png">

Peacock is the last name of the employee with the most orders.

**c. What product was ordered the most by customers in Germany?**

SQL Query: SELECT TOP 1 P.ProductID, ProductName, SUM(Quantity) as Quantity FROM  OrderDetails OD, Orders O, Products P WHERE OD.OrderID = O.OrderID and P.ProductID = OD.ProductID and O.CustomerID IN (SELECT CustomerID FROM Customers C WHERE C.Country = 'Germany') GROUP BY P.ProductID, ProductName ORDER BY SUM(Quantity) DESC;

Boston Crab Meat was being ordered the most and have a total sale of 160 units by customers in Germany. 

<img src="files/c.png">